In [1]:
# Dataset - https://www.iea.org/data-and-statistics/data-product/monthly-oil-price-statistics-2#data-sets
# Use sheet labeled "raw"

import pandas as pd

# Read in the data from the Excel sheet
df = pd.read_excel('input.xlsx')

# Convert the TIME column to a pandas datetime object
df['TIME'] = pd.to_datetime(df['TIME'], format='%b-%y')

# Group the data by COUNTRY and PRODUCT and sort by TIME
df_grouped = df.groupby(['COUNTRY', 'PRODUCT']).apply(lambda x: x.sort_values('TIME'))

# Reset the index of the grouped dataframe
df_grouped = df_grouped.reset_index(drop=True)

# Print the preprocessed data
print(df_grouped)

            COUNTRY                PRODUCT         FLOW        UNIT  \
0           Austria    Diesel (unit/litre)  Total Price  US dollars   
1           Austria    Diesel (unit/litre)  Total Price  US dollars   
2           Austria    Diesel (unit/litre)  Total Price  US dollars   
3           Austria    Diesel (unit/litre)  Total Price  US dollars   
4           Austria    Diesel (unit/litre)  Total Price  US dollars   
...             ...                    ...          ...         ...   
9434  United States  Gasoline (unit/litre)  Total Price  US dollars   
9435  United States  Gasoline (unit/litre)  Total Price  US dollars   
9436  United States  Gasoline (unit/litre)  Total Price  US dollars   
9437  United States  Gasoline (unit/litre)  Total Price  US dollars   
9438  United States  Gasoline (unit/litre)  Total Price  US dollars   

           TIME  VALUE  
0    2015-01-01   1.28  
1    2015-02-01   1.29  
2    2015-03-01   1.26  
3    2015-04-01   1.26  
4    2015-05-01   1.32

In [2]:
import numpy as np

def monte_carlo_sim(df, country, product, num_simulations=1000000, num_periods=12):
    # Filter the dataframe for the specified country and product
    df_filtered = df[(df['COUNTRY'] == country) & (df['PRODUCT'] == product)]
    
    # Return None for all outputs if the filtered dataframe is empty
    if df_filtered.empty:
        return None, None, None
    
    # Get the most recent value
    last_value = df_filtered.iloc[-1]['VALUE']
    
    # Calculate the monthly percent change in the value
    monthly_returns = df_filtered['VALUE'].pct_change()
    monthly_returns = monthly_returns.dropna()
    
    # Calculate the mean and standard deviation of the monthly returns
    mean_return = monthly_returns.mean()
    std_return = monthly_returns.std()
    
    # Generate the random monthly returns for the simulations
    returns = np.random.normal(mean_return, std_return, (num_simulations, num_periods))
    
    # Calculate the future values for each simulation
    future_values = np.zeros((num_simulations, num_periods))
    future_values[:, 0] = last_value * (1 + returns[:, 0])
    for i in range(1, num_periods):
        future_values[:, i] = future_values[:, i-1] * (1 + returns[:, i])
    
    # Calculate the mean, lower 95%, and upper 95% predictions
    mean_prediction = np.mean(future_values, axis=0)
    lower_prediction_95 = np.percentile(future_values, 5, axis=0)
    upper_prediction_95 = np.percentile(future_values, 95, axis=0)
    lower_prediction_75 = np.percentile(future_values, 25, axis=0)
    upper_prediction_75 = np.percentile(future_values, 75, axis=0)
    
    return mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75, upper_prediction_75


In [3]:
import plotly.graph_objs as go

def plot_forecast(df, country, product, mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75,upper_prediction_75):
    # Filter the dataframe for the specified country and product
    df_filtered = df[(df['COUNTRY'] == country) & (df['PRODUCT'] == product)]
    
    if df_filtered.empty:
        print(f"No data available for {product} - {country}")
        return
    
    # Extract the start and end dates from the data
    start_date = df_filtered['TIME'].min()
    end_date = df_filtered['TIME'].max()
    
    # Create the actual data trace
    actual_trace = go.Scatter(x=df_filtered['TIME'], y=df_filtered['VALUE'], name='Actual Data', mode='lines')
    
    # Create the mean prediction trace
    mean_trace = go.Scatter(x=pd.date_range(start=end_date, periods=len(mean_prediction), freq='M'), 
                            y=mean_prediction, name='Mean Prediction', mode='lines')
    
    # Create the lower prediction trace 5th percentile
    lower_trace_95 = go.Scatter(x=pd.date_range(start=end_date, periods=len(lower_prediction_95), freq='M'), 
                             y=lower_prediction_95, name='Lower Prediction 95%', mode='lines', line=dict(dash='dash'))
    
    # Create the upper prediction trace 95th percentile
    upper_trace_95 = go.Scatter(x=pd.date_range(start=end_date, periods=len(upper_prediction_95), freq='M'), 
                             y=upper_prediction_95, name='Upper Prediction 95%', mode='lines', line=dict(dash='dash'))
    
    # Create the lower prediction trace 25th percentile
    lower_trace_75 = go.Scatter(x=pd.date_range(start=end_date, periods=len(lower_prediction_75), freq='M'), 
                             y=lower_prediction_75, name='Lower Prediction 75%', mode='lines', line=dict(dash='dash'))
    
    # Create the upper prediction trace 75th percentile
    upper_trace_75 = go.Scatter(x=pd.date_range(start=end_date, periods=len(upper_prediction_75), freq='M'), 
                             y=upper_prediction_75, name='Upper Prediction 75%', mode='lines', line=dict(dash='dash'))
    
    # Create the plot
    fig = go.Figure([actual_trace, mean_trace, lower_trace_95, upper_trace_95, lower_trace_75, upper_trace_75])
    
    # Set the plot title and axis labels
    fig.update_layout(title=f'{product} - {country} Forecast', xaxis_title='Time', yaxis_title='Price (US dollars)')
    
    # Show the plot
    fig.show()


In [4]:
# Loop through each product and country
for country in df_grouped['COUNTRY'].unique():
    for product in df_grouped['PRODUCT'].unique():
        try:
            # Perform the Monte Carlo simulation
            mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75,upper_prediction_75 = monte_carlo_sim(df_grouped, country, product)
            
            # Generate the plot
            plot_forecast(df_grouped, country, product, mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75,upper_prediction_75)
        except:
            print(f"Error plotting forecast for {product} - {country}")

Error plotting forecast for Domestic heating oil (unit/litre) - Brazil


Error plotting forecast for Domestic heating oil (unit/litre) - Slovak Republic
